In [1]:
# import osmnx as ox
import pandas as pd
import geopandas as gpd
import networkx as nx

import warnings
import sys
import os
warnings.filterwarnings("ignore")
sys.stderr = open(os.devnull, 'w')

# local crs
local_crs = 32636

In [2]:
adj_mx = pd.read_pickle('/Users/mvin/Code/PopFrame/examples/data/model_data/adj_mx.pickle')
towns = gpd.read_parquet('/Users/mvin/Code/PopFrame/examples/data/model_data/towns.parquet')
okrugs = gpd.read_parquet('/Users/mvin/Code/PopFrame/examples/data/model_data/okrugs.parquet')
rayons = gpd.read_parquet('/Users/mvin/Code/PopFrame/examples/data/model_data/rayons.parquet')
territory = gpd.read_parquet('/Users/mvin/Code/PopFrame/examples/data/model_data/territory.parquet')


In [3]:
from popframe.models.region import Region

region = Region(
  towns=towns,
  okrugs=okrugs, 
  rayons=rayons, 
  adjacency_matrix=adj_mx,
  territory=territory
)

In [4]:
profiles = {
    "Жилая застройка - ИЖС": {
        "criteria": {"Население": 1, "Транспорт": 2, "Экология": 4, "Соц-об": 4, "Инж инф": 3, "Кул-дос-сп": 0},
        "weights": {"Население": 0, "Транспорт": 0, "Экология": 1, "Соц-об": 1, "Инж инф": 1, "Кул-дос-сп": 0}
    },
    "Жилая застройка - Малоэтажная": {
        "criteria": {"Население": 3, "Транспорт": 3, "Экология": 4, "Соц-об": 5, "Инж инф": 4, "Кул-дос-сп": 1},
        "weights": {"Население": 0, "Транспорт": 0, "Экология": 1, "Соц-об": 1, "Инж инф": 1, "Кул-дос-сп": 0}
    },
    "Жилая застройка - Среднеэтажная": {
        "criteria": {"Население": 4, "Транспорт": 4, "Экология": 4, "Соц-об": 4, "Инж инф": 5, "Кул-дос-сп": 2},
        "weights": {"Население": 1, "Транспорт": 1, "Экология": 1, "Соц-об": 1, "Инж инф": 1, "Кул-дос-сп": 0}
    },
    "Жилая застройка - Многоэтажная": {
        "criteria": {"Население": (4, 5), "Транспорт": 5, "Экология": 4, "Соц-об": 5, "Инж инф": 5, "Кул-дос-сп": 3},
        "weights": {"Население": 1, "Транспорт": 1, "Экология": 1, "Соц-об": 1, "Инж инф": 1, "Кул-дос-сп": 0}
    },
    "Общественно-деловая": {
        "criteria": {"Население": 4, "Транспорт": 5, "Экология": 4, "Соц-об": 2, "Инж инф": 5, "Кул-дос-сп": 3},
        "weights": {"Население": 1, "Транспорт": 1, "Экология": 1, "Соц-об": 0, "Инж инф": 1, "Кул-дос-сп": 0}
    },
    "Рекреационная": {
        "criteria": {"Население": 0, "Транспорт": 0, "Экология": 0, "Соц-об": 0, "Инж инф": 0, "Кул-дос-сп": 0},
        "weights": {"Население": 0, "Транспорт": 0, "Экология": 0, "Соц-об": 0, "Инж инф": 0, "Кул-дос-сп": 0}
    },
    "Специального назначения - медицинский центр": {
        "criteria": {"Население": 4, "Транспорт": 4, "Экология": 5, "Соц-об": 0, "Инж инф": 5, "Кул-дос-сп": 0},
        "weights": {"Население": 1, "Транспорт": 1, "Экология": 1, "Соц-об": 0, "Инж инф": 1, "Кул-дос-сп": 0}
    },
    "Специального назначения - туристический кластер": {
        "criteria": {"Население": 3, "Транспорт": 3, "Экология": 5, "Соц-об": 1, "Инж инф": 3, "Кул-дос-сп": 0},
        "weights": {"Население": 0, "Транспорт": 0, "Экология": 1, "Соц-об": 0, "Инж инф": 0, "Кул-дос-сп": 0}
    },
    "Промышленная": {
        "criteria": {"Население": (3, 4), "Транспорт": 4, "Экология": 0, "Соц-об": 1, "Инж инф": 4, "Кул-дос-сп": 2},
        "weights": {"Население": 1, "Транспорт": 1, "Экология": 0, "Соц-об": 0, "Инж инф": 1, "Кул-дос-сп": 0}
    },
    "Сельско-хозяйственная": {
        "criteria": {"Население": (2, 3), "Транспорт": 3, "Экология": (4, 5), "Соц-об": 3, "Инж инф": (2, 3), "Кул-дос-сп": 2},
        "weights": {"Население": 0, "Транспорт": 0, "Экология": 1, "Соц-об": 0, "Инж инф": 0, "Кул-дос-сп": 0}
    },
    "Транспортная инженерная": {
        "criteria": {"Население": (1, 2), "Транспорт": 0, "Экология": 0, "Соц-об": 0, "Инж инф": 2, "Кул-дос-сп": 0},
        "weights": {"Население": 0, "Транспорт": 0, "Экология": 0, "Соц-об": 0, "Инж инф": 0, "Кул-дос-сп": 0}
    }
}

# Пример использования:
criteria_values = {
    "Население": 4,
    "Транспорт": 3,
    "Экология": 5,
    "Соц-об": 4,
    "Инж инф": 4,
    "Кул-дос-сп": 3
}



In [5]:
from popframe.method.territory_evaluation import TerritoryEvaluation

evaluation = TerritoryEvaluation(region=region)
ranked_profiles = evaluation.calculate_potential(profiles, criteria_values)
for profile, (potential, weighted_score) in ranked_profiles:
    print(f"Профиль: {profile}, Потенциал: {potential}, Соответствий важным критериям: {weighted_score}")


Профиль: Жилая застройка - ИЖС, Потенциал: 6, Соответствий важным критериям: 2
Профиль: Специального назначения - туристический кластер, Потенциал: 6, Соответствий важным критериям: 0
Профиль: Жилая застройка - Малоэтажная, Потенциал: 5, Соответствий важным критериям: 1
Профиль: Промышленная, Потенциал: 5, Соответствий важным критериям: 1
Профиль: Транспортная инженерная, Потенциал: 5, Соответствий важным критериям: 0
Профиль: Жилая застройка - Среднеэтажная, Потенциал: 4, Соответствий важным критериям: 1
Профиль: Общественно-деловая, Потенциал: 4, Соответствий важным критериям: 1
Профиль: Сельско-хозяйственная, Потенциал: 4, Соответствий важным критериям: 1
Профиль: Специального назначения - медицинский центр, Потенциал: 4, Соответствий важным критериям: 0
Профиль: Жилая застройка - Многоэтажная, Потенциал: 3, Соответствий важным критериям: 1
Профиль: Рекреационная, Потенциал: 6, Соответствий важным критериям: 0
